# Notebook Imports

In [1]:
import pandas as pd
import numpy as np

# Constants

In [75]:
TRAINING_DATA_FILE = 'SpamData/02_Training/train-data.txt'
TEST_DATA_FILE = 'SpamData/02_Training/test-data.txt'

TOKEN_SPAM_PROB_FILE = 'SpamData/03_Testing/prob-spam.txt'
TOKEN_HAM_PROB_FILE = 'SpamData/03_Testing/prob-nonspam.txt'
TOKEN_ALL_PROB_FILE = 'SpamData/03_Testing/prob-all-tokens.txt'

TEST_FEATURE_MATRIX = 'SpamData/03_Testing/test-features.txt'
TEST_TARGET_FILE = 'SpamData/03_Testing/test-target.txt'

VOCAB_SIZE = 2500

# Read and load features from .txt files into numpy array

In [3]:
sparse_train_data = np.loadtxt(TRAINING_DATA_FILE, delimiter=' ', dtype=int)

In [4]:
sparse_test_data = np.loadtxt(TEST_DATA_FILE, delimiter=' ', dtype=int)

In [5]:
sparse_test_data[:5]

array([[8, 2, 1, 1],
       [8, 3, 1, 4],
       [8, 4, 1, 2],
       [8, 5, 1, 1],
       [8, 6, 1, 2]])

In [7]:
sparse_train_data[-5:]

array([[5795, 2035,    0,    2],
       [5795, 2040,    0,    1],
       [5795, 2272,    0,    1],
       [5795, 2342,    0,    1],
       [5795, 2370,    0,    1]])

In [10]:
print('Nr of rows in training file:', sparse_train_data.shape[0])
print('Nr of rows in testing file:', sparse_test_data.shape[0])

Nr of rows in training file: 258360
Nr of rows in testing file: 117776


In [14]:
print('Nr of unique emails in training file:', np.unique(sparse_train_data[:, 0]).size)
print('Nr of unique emails in testing file:', np.unique(sparse_test_data[:, 0]).size)

Nr of unique emails in training file: 4013
Nr of unique emails in testing file: 1724


## How to create an empty DataFrame

In [15]:
column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
column_names[:5]

['DOC_ID', 'CATEGORY', 0, 1, 2]

In [16]:
len(column_names)

2502

In [17]:
index_names = np.unique(sparse_train_data[:, 0])
index_names

array([   0,    1,    2, ..., 5791, 5794, 5795])

In [23]:
full_train_data = pd.DataFrame(index=index_names, columns=column_names)
full_train_data.fillna(value=0, inplace=True)

In [24]:
full_train_data

,DOC_ID,CATEGORY,0,1,2,3,4,5,6,7,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5794,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Create a full matrix from a sparse matrix

In [25]:
def make_full_matrix(sparse_matrix, nr_words, doc_idx=0, word_idx=1, cat_idx=2, freq_idx=3):
    """
    Form a full matrix from a sparse matrix. Return a pandas DataFrame.
    Keyword arguements:
    spare_matrix -- numpy array
    nr_words -- size of the vocabulary. Total number of tokens.
    doc_idx -- position of the document id in the sparse matrix. Default: 1st column
    word_idx -- position of the word id in the sparse matrix. Default: 2nd column
    cat_idx -- position of the label(spam: 1, ham: 0). Default: 3rd column
    freq_idx -- position of the occurrence of word in sparse matrix. Default: 4th column
    """
    
    column_names = ['DOC_ID'] + ['CATEGORY'] + list(range(0, VOCAB_SIZE))
    doc_id_names = np.unique(sparse_matrix[:, 0])
    full_matrix = pd.DataFrame(index=doc_id_names, columns=column_names)
    full_matrix.fillna(value=0, inplace=True)
    
    for i in range(sparse_matrix.shape[0]):
        doc_nr = sparse_matrix[i][doc_idx]
        word_id = sparse_matrix[i][word_idx]
        label = sparse_matrix[i][cat_idx]
        occurrence = sparse_matrix[i][freq_idx]
        
        full_matrix.at[doc_nr, 'DOC_ID'] = doc_nr
        full_matrix.at[doc_nr, 'CATEGORY'] = label
        full_matrix.at[doc_nr, word_id] = occurrence
    
    full_matrix.set_index('DOC_ID', inplace=True)
    
    return full_matrix

In [26]:
%%time

full_train_data = make_full_matrix(sparse_train_data, VOCAB_SIZE)

CPU times: total: 25.8 s
Wall time: 26.8 s


In [27]:
full_train_data.head()

,CATEGORY,0,1,2,3,4,5,6,7,8,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,1,0,0,1,2,1,0,0,3,0,...,0,0,0,0,0,0,0,0,0,0
1,1,7,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,1,6,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,6,0,0,2,4,0,3,14,0,...,0,0,0,0,0,0,0,0,0,0
4,1,5,1,2,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


# Training the Naive Bayes Model
## Calculating the probabilities of spam

In [29]:
full_train_data.CATEGORY.size

4013

In [37]:
prob_spam = full_train_data.CATEGORY.sum() / full_train_data.CATEGORY.size
# spam has category 1
print('Probability of spam is:', prob_spam)

Probability of spam is: 0.310989284824321


# Total number of words / tokens

In [40]:
full_train_features = full_train_data.loc[:, full_train_data.columns != 'CATEGORY']
full_train_features

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
0,0,0,1,2,1,0,0,3,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,1,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,0,0,2,4,0,3,14,0,0,...,0,0,0,0,0,0,0,0,0,0
4,5,1,2,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [39]:
email_lengths = full_train_features.sum(axis=1)
email_lengths.shape

(4013,)

In [41]:
email_lengths[:5]

DOC_ID
0     87
1     53
2     40
3    183
4     43
dtype: int64

In [42]:
total_wc = email_lengths.sum()
total_wc

429241

## Number of tokens in spam and ham emails

In [43]:
spam_lengths = email_lengths[full_train_data.CATEGORY == 1]
spam_lengths.shape

(1248,)

In [53]:
non_spam_lengths = email_lengths[full_train_data.CATEGORY == 0]
non_spam_lengths.shape

(2765,)

In [45]:
spam_wc = spam_lengths.sum()
spam_wc

176334

In [47]:
non_spam_wc = total_wc - spam_wc
non_spam_wc

252907

In [54]:
print('Average nr of word in spam emails: {:.0f}' .format(spam_wc/spam_lengths.shape[0]))
print('Average nr of word in ham emails: {:.0f}'.format(non_spam_wc/non_spam_lengths.shape[0]))

Average nr of word in spam emails: 141
Average nr of word in ham emails: 91


## Summing the tokens occurring in spam

In [55]:
full_train_features.shape

(4013, 2500)

In [57]:
train_spam_tokens = full_train_features.loc[full_train_data.CATEGORY == 1]
train_spam_tokens.head()
train_spam_tokens.tail()

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1885,1,0,0,2,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1887,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1889,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1890,2,0,0,0,1,0,0,2,0,0,...,0,0,0,0,0,0,0,0,0,0
1895,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
train_spam_tokens.shape

(1248, 2500)

In [60]:
summed_spam_tokens = train_spam_tokens.sum(axis=0) + 1
summed_spam_tokens

0       2179
1        935
2       1217
3       2022
4       1219
        ... 
2495       3
2496       2
2497      17
2498      15
2499       2
Length: 2500, dtype: int64

In [62]:
train_ham_tokens = full_train_features.loc[full_train_data.CATEGORY == 0]
train_ham_tokens

,0,1,2,3,4,5,6,7,8,9,...,2490,2491,2492,2493,2494,2495,2496,2497,2498,2499
DOC_ID,,,,,,,,,,,,,,,,,,,,,
1896,1,2,1,0,1,1,2,0,1,1,...,0,0,0,0,0,0,0,0,0,0
1898,2,1,0,1,1,0,2,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1899,2,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1900,3,4,0,1,0,0,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1901,2,2,0,1,0,0,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5789,3,1,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
5790,1,1,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5791,3,1,0,1,1,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0


In [63]:
summed_ham_tokens = train_ham_tokens.sum(axis=0) + 1
summed_ham_tokens

0       5484
1       2590
2       2045
3        938
4       1612
        ... 
2495      27
2496      35
2497      20
2498      15
2499      28
Length: 2500, dtype: int64

In [65]:
train_ham_tokens[0].sum() + 1

5484

# P(token | spam) - Probability that a token occurs given the email is spam

In [67]:
prob_tokens_spam = summed_spam_tokens / (spam_wc + VOCAB_SIZE)
prob_tokens_spam[:5]

0    0.012184
1    0.005228
2    0.006805
3    0.011307
4    0.006816
dtype: float64

In [68]:
prob_tokens_spam.sum()

1.0

# P(token | ham) - Probability that a token occurs given the email is non-spam

In [69]:
prob_tokens_nonspam = summed_ham_tokens / (non_spam_wc + VOCAB_SIZE)
prob_tokens_nonspam[:5]

0    0.021472
1    0.010141
2    0.008007
3    0.003673
4    0.006311
dtype: float64

In [70]:
prob_tokens_nonspam.sum()

1.0

# P(token) - Probability that a token occurs

In [71]:
prob_tokens_all = full_train_features.sum(axis=0) / total_wc

In [72]:
prob_tokens_all.sum()

1.0

# Save the train model

In [74]:
np.savetxt(TOKEN_SPAM_PROB_FILE, prob_tokens_spam)
np.savetxt(TOKEN_HAM_PROB_FILE, prob_tokens_nonspam)
np.savetxt(TOKEN_ALL_PROB_FILE, prob_tokens_all)

# Prepare test data

In [76]:
sparse_test_data.shape

(117776, 4)

In [77]:
%%time

full_test_data = make_full_matrix(sparse_test_data, nr_words=VOCAB_SIZE)

CPU times: total: 12.2 s
Wall time: 12.3 s


In [79]:
X_test = full_test_data.loc[:, full_test_data.columns != 'CATEGORY']
y_test = full_test_data.CATEGORY

In [80]:
np.savetxt(TEST_TARGET_FILE, y_test)
np.savetxt(TEST_FEATURE_MATRIX, X_test)